In [1]:
import numpy as np
import scipy as scp
import sympy as sp
import matplotlib.pyplot as plt

c0=299792458.


In [2]:
#Brechungsindex für Nlak21
lambda_ = sp.Symbol('lambda')

n_ = sp.sqrt(1+(1.22718116*lambda_**2)/(lambda_**2-0.00602075682)
                   +(0.420783743*lambda_**2)/(lambda_**2-0.0196862889)
                   +(1.01284843*lambda_**2)/(lambda_**2-88.4370099))

dn_ = sp.diff(n_,lambda_)
ddn_ = sp.diff(dn_,lambda_)

#numpy funktionen erstellen
n_prism = sp.lambdify(lambda_, n_, modules='numpy')
dn_prism = sp.lambdify(lambda_, dn_, modules='numpy')
ddn_prism = sp.lambdify(lambda_, ddn_, modules='numpy')

In [3]:
#prism gdd Formel
def prism_gdd(lambdal,n,dn,ddn,L,insertion=2.5e-3):
    Lg=2*insertion*1/np.sqrt(1+n**2)
    result = 2*lambdal**3*(2*np.pi*c0**2)**(-1)*(Lg*ddn-(4*L+Lg*n**(-3))*(dn)**2)
    return result

In [4]:
lambdal_mikro=0.8
lambdal_si=800e-9 #carrier wavelength
L=432.0e-3 #Prism separation (measured)

print prism_gdd(lambdal_si,n_prism(lambdal_mikro),dn_prism(lambdal_mikro)*1e6,ddn_prism(lambdal_mikro)*1e12,L,2.5e-3)



-1.67775581848e-27


In [33]:
# prism compr auswertung (gdd in abhängigkeit von insertion:)
insarr=np.array([2.84e-3,2.5e-3,2.04e-3,1.75e-3,1.3e-3,0.7e-3,0])

gddarr=prism_gdd(lambdal_si,n_prism(lambdal_mikro),dn_prism(lambdal_mikro)*1e6,ddn_prism(lambdal_mikro)*1e12,L,insarr)

print gddarr

result_pc=np.column_stack((insarr,gddarr))
np.savetxt('pc.txt',result_pc)
plt.plot(insarr,gddarr)
plt.show()

[ -1.63423469e-27  -1.67775582e-27  -1.73663734e-27  -1.77375830e-27
  -1.83135979e-27  -1.90816177e-27  -1.99776409e-27]


In [139]:
#pc hwhm:
pcarr = np.loadtxt('pc_fwhm.dat')

apparent_bw=8e-9*2*np.pi*c0*lambdal_si**(-2)

#ausgangspulslänge vom laser:

def timewidth(t0,gdd,bandwidth):
    return np.sqrt(t0**2+gdd**2*bandwidth**2) #Faktor ????
def timewidth0(t,gdd,bandwidth):
    #return np.sqrt(np.absolute(t**2-gdd**2*bandwidth**2)) #Faktor ????
    return np.sqrt(t**2-gdd**2*bandwidth**2) #Faktor ????

t0arr = pcarr[:,1] #temp. gemessen (FWHM)
#bandwidtharr = pcarr[:,2]*2*np.pi*c0*lambdal_si**(-2)
bandwidtharr = pcarr[:,2]*2*np.pi*c0*lambdal_si**(-2) #Bandbreite in lambda

gdd_opt = gddarr[0] #gdd bei opt. einstellung (berechnet mit pc formel)


#timewidth_opt = timewidth(t0arr[0],gddarr[0],bandwidtharr[0]) #was aus dem laser rauskommen sollte.
timewidth_opt = timewidth(t0arr[0],gddarr[0],apparent_bw)

timewidth_theory_arr = timewidth0(timewidth_opt,gddarr,apparent_bw)


/usr/lib/python2.7/site-packages/IPython/kernel/__main__.py:12: RuntimeWarning: invalid value encountered in sqrt


In [121]:
print timewidth_theory_arr

[  1.40672962e-13   1.53512080e-13   1.69806127e-13   1.79579332e-13
   1.94139771e-13   2.12640758e-13   2.33210090e-13]


In [122]:
print pcarr[:,1]

[  2.45000000e-14   2.56000000e-14   2.89000000e-14   3.57000000e-14
   4.56000000e-14   5.33000000e-14   5.66000000e-14]


In [125]:
bandwidtharr

array([  1.36270418e+14,   1.38624983e+14,   1.73943449e+14,
         1.42451150e+14,   1.45982996e+14,   1.29795366e+14,
         1.19788467e+14])

In [127]:
print 55e-9*2*np.pi*c0*lambdal_si**(-2)

1.61876306566e+14
